# Prepare training container

In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-2.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 553020858742.dkr.ecr.us-east-2.amazonaws.com

In [ ]:
! pygmentize -l docker Dockerfile

In [ ]:
! ./build_and_push.sh hf-transformers-sm latest Dockerfile

# Training on SM

In [6]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [7]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

In [8]:
container = "hf-transformers-sm" # your container name
tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container, tag)

In [11]:
# placeholder
metric_definitions=[]


In [12]:
#placeholder
hyperparameters = {}

In [ ]:
d2 = sagemaker.estimator.Estimator(image,
                                   role=role,
                                   train_instance_count=1, 
                                   train_instance_type='ml.p3.2xlarge',
#                                   train_instance_type="local_gpu", # use local_gpu for quick troubleshooting
#                                   train_volume_size=100,
                                   output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                   metric_definitions = metric_definitions,
                                   hyperparameters = hyperparameters, 
                                   sagemaker_session=sess)

d2.fit() 

2020-05-18 22:07:52 Starting - Starting the training job...
2020-05-18 22:07:54 Starting - Launching requested ML instances......
2020-05-18 22:08:57 Starting - Preparing the instances for training......
2020-05-18 22:10:16 Downloading - Downloading input data...
2020-05-18 22:10:25 Training - Downloading the training image...............
2020-05-18 22:13:20 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-18 22:13:10,660 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-18 22:13:10,685 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-18 22:13:10,688 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-18 22:13:11,222 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-05-18 22:13:11,22